# MGP Building Segmentation Pipeline

**Municipal GIS Partners • Spark Initiative**

---

Interactive building footprint extraction using SAM3 (Segment Anything Model 3) for municipal orthoimagery.

This notebook implements MGP's standardized workflow for generating GIS-ready building polygons in Illinois State Plane coordinates.

## Pipeline Overview

1. **Load Imagery** → Georeferenced orthophoto (GeoTIFF)
2. **Verify/Reproject CRS** → Illinois State Plane East (EPSG:6455)
3. **Tile Large Rasters** → 2048×2048 px for optimal detection
4. **SAM3 Segmentation** → Interactive or text-prompted extraction
5. **Vectorize & Clean** → Export to GIS formats (GPKG, SHP, GDB)

---
## 1. Installation

In [ ]:
# Install segment-geospatial with SAM3 support
# %pip install "segment-geospatial[samgeo3]"

# For latest transformers (required for SAM3)
# %pip install git+https://github.com/huggingface/transformers.git

# Additional GIS tools
# %pip install rasterio fiona geopandas pyproj

---
## 2. Import Libraries

In [ ]:
import os
from pathlib import Path
from datetime import datetime

import leafmap
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import Window
import geopandas as gpd
from pyproj import CRS

from samgeo import SamGeo3, download_file

---
## 3. Configuration

Set project parameters for your municipality.

In [ ]:
# ============================================================
# PROJECT CONFIGURATION - EDIT FOR EACH MUNICIPALITY
# ============================================================

# Municipality / Project Name
MUNICIPALITY = "Sample_Community"
PROJECT_DATE = datetime.now().strftime("%Y%m%d")

# Input/Output Paths
INPUT_IMAGE = "path/to/orthoimagery.tif"  # Your input raster
OUTPUT_DIR = Path(f"./outputs/{MUNICIPALITY}_{PROJECT_DATE}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Illinois State Plane East (US Survey Feet)
# Standard CRS for Cook County and northeastern Illinois municipalities
TARGET_CRS = "EPSG:6455"  # NAD83(2011) / Illinois East (ftUS)

# Tile size for SAM3 processing
# 2048x2048 is optimal for detecting residential buildings
# Smaller tiles = better small building detection
TILE_SIZE = 2048

# Minimum building area in square feet (filters noise)
MIN_BUILDING_AREA_SQFT = 100

print(f"Project: {MUNICIPALITY}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Target CRS: {TARGET_CRS}")

---
## 4. Imagery Sources Reference

### Cook County Orthoimagery
- **Portal**: [Cook Central](https://hub-cookcountyil.opendata.arcgis.com/)
- **Viewer**: [CookViewer](https://maps.cookcountyil.gov/cookviewer/)
- **Resolution**: 0.5 ft (6 inch) pixel resolution
- **Bands**: 4-band (R, G, B, NIR)
- **Native CRS**: Illinois State Plane East (US Survey Feet)

### Other Regional Sources
- **Illinois Geospatial Data Clearinghouse**: [clearinghouse.isgs.illinois.edu](https://clearinghouse.isgs.illinois.edu)
- **USDA NAIP**: Available via Geospatial Data Gateway
- **Municipal imagery flights**: Contact individual communities

### Requirements
- Minimum resolution: ≤0.3 meters per pixel (1 ft or better)
- Format: GeoTIFF with embedded georeferencing
- Full coverage of project area

---
## 5. Load and Inspect Imagery

In [ ]:
# For demonstration, download sample imagery
# Replace with your municipal orthoimagery path

DEMO_MODE = True  # Set to False when using real data

if DEMO_MODE:
    url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley.tif"
    INPUT_IMAGE = download_file(url)
    print(f"Demo imagery downloaded: {INPUT_IMAGE}")

In [ ]:
def inspect_raster(image_path):
    """Display raster metadata and CRS information."""
    with rasterio.open(image_path) as src:
        print("=" * 60)
        print("RASTER METADATA")
        print("=" * 60)
        print(f"File: {Path(image_path).name}")
        print(f"Dimensions: {src.width} x {src.height} pixels")
        print(f"Bands: {src.count}")
        print(f"Data Type: {src.dtypes[0]}")
        print(f"\nCRS: {src.crs}")
        print(f"CRS (WKT): {src.crs.wkt[:100]}..." if src.crs else "No CRS defined!")
        print(f"\nBounds: {src.bounds}")
        print(f"Resolution: {src.res[0]:.4f} x {src.res[1]:.4f} {src.crs.linear_units if src.crs else 'units'}")
        print(f"\nNoData Value: {src.nodata}")
        
        # Check if reprojection needed
        if src.crs:
            current_epsg = src.crs.to_epsg()
            target_epsg = int(TARGET_CRS.split(":")[1])
            if current_epsg == target_epsg:
                print(f"\n✓ CRS matches target ({TARGET_CRS})")
            else:
                print(f"\n⚠ CRS mismatch: Current {current_epsg} → Target {target_epsg}")
                print("  Reprojection required before processing")
        else:
            print("\n⚠ No CRS defined - georeferencing required!")
        
        return src.crs, src.bounds, (src.width, src.height)

crs_info, bounds_info, dims = inspect_raster(INPUT_IMAGE)

In [ ]:
# Visualize input imagery
m = leafmap.Map()
m.add_raster(INPUT_IMAGE, layer_name="Input Orthoimagery")
m

---
## 6. CRS Standardization

All imagery must be projected to **Illinois State Plane East (EPSG:6455)** for:
- Pixel-accurate spatial alignment with MGP enterprise GIS
- Correct area and distance calculations in US Survey Feet
- Seamless integration with municipal parcel and infrastructure data

In [ ]:
def reproject_raster(input_path, output_path, target_crs="EPSG:6455"):
    """
    Reproject raster to Illinois State Plane East.
    
    Parameters:
        input_path: Path to input GeoTIFF
        output_path: Path for reprojected output
        target_crs: Target coordinate reference system
    
    Returns:
        Path to reprojected raster
    """
    with rasterio.open(input_path) as src:
        # Skip if already in target CRS
        if src.crs and src.crs.to_epsg() == int(target_crs.split(":")[1]):
            print(f"✓ Already in {target_crs}, no reprojection needed")
            return input_path
        
        # Calculate transform for target CRS
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )
        
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        print(f"Reprojecting to {target_crs}...")
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.bilinear
                )
        
        print(f"✓ Reprojected: {output_path}")
        return output_path

In [ ]:
# Reproject if needed (skip for demo - it's already georeferenced)
# For production, uncomment and run:

# reprojected_image = OUTPUT_DIR / f"{MUNICIPALITY}_reprojected.tif"
# INPUT_IMAGE = reproject_raster(INPUT_IMAGE, reprojected_image, TARGET_CRS)

---
## 7. Raster Tiling (Critical for Building Detection)

**Why tile?** SAM3 suppresses small buildings when scene context is too large. 

Standard tile size: **2048 × 2048 pixels**

This dramatically improves detection of:
- Single-family homes
- Small commercial structures  
- Detached garages and accessory buildings

In [ ]:
def tile_raster(input_path, output_dir, tile_size=2048):
    """
    Split large raster into tiles for optimal SAM3 processing.
    
    Parameters:
        input_path: Path to input GeoTIFF
        output_dir: Directory for tile outputs
        tile_size: Tile dimensions in pixels (default 2048)
    
    Returns:
        List of tile file paths
    """
    tiles_dir = Path(output_dir) / "tiles"
    tiles_dir.mkdir(parents=True, exist_ok=True)
    
    tile_paths = []
    
    with rasterio.open(input_path) as src:
        # Check if tiling is needed
        if src.width <= tile_size and src.height <= tile_size:
            print(f"✓ Image ({src.width}x{src.height}) fits in single tile, no splitting needed")
            return [input_path]
        
        n_tiles_x = (src.width + tile_size - 1) // tile_size
        n_tiles_y = (src.height + tile_size - 1) // tile_size
        total_tiles = n_tiles_x * n_tiles_y
        
        print(f"Tiling {src.width}x{src.height} image into {n_tiles_x}x{n_tiles_y} = {total_tiles} tiles")
        
        for row in range(n_tiles_y):
            for col in range(n_tiles_x):
                # Calculate window
                x_off = col * tile_size
                y_off = row * tile_size
                width = min(tile_size, src.width - x_off)
                height = min(tile_size, src.height - y_off)
                
                window = Window(x_off, y_off, width, height)
                
                # Read tile data
                tile_data = src.read(window=window)
                
                # Skip empty tiles
                if tile_data.max() == 0:
                    continue
                
                # Update transform for tile
                tile_transform = src.window_transform(window)
                
                # Output path
                tile_name = f"tile_{row:03d}_{col:03d}.tif"
                tile_path = tiles_dir / tile_name
                
                # Write tile
                kwargs = src.meta.copy()
                kwargs.update({
                    'width': width,
                    'height': height,
                    'transform': tile_transform
                })
                
                with rasterio.open(tile_path, 'w', **kwargs) as dst:
                    dst.write(tile_data)
                
                tile_paths.append(tile_path)
        
        print(f"✓ Created {len(tile_paths)} tiles in {tiles_dir}")
        return tile_paths

In [ ]:
# Tile the imagery
tile_paths = tile_raster(INPUT_IMAGE, OUTPUT_DIR, TILE_SIZE)
print(f"\nTiles ready for processing: {len(tile_paths)}")

---
## 8. Hugging Face Authentication

SAM3 requires access approval from Meta AI via Hugging Face.

**One-time setup:**
1. Request access at: https://huggingface.co/facebook/sam3
2. Wait for approval (usually < 24 hours)
3. Run the login cell below

In [ ]:
# Uncomment and run once to authenticate
# from huggingface_hub import login
# login()

---
## 9. Initialize SAM3

In [ ]:
# Initialize SAM3 with transformers backend
sam3 = SamGeo3(
    backend="transformers",
    device=None,  # Auto-detect GPU/CPU
    checkpoint_path=None,
    load_from_HF=True
)

print("✓ SAM3 initialized")

---
## 10. Interactive Segmentation (Production Standard)

**Interactive prompting** is the MGP production standard.

Methods:
- **Text prompt**: "building" for automatic detection
- **Point prompts**: Click on buildings to segment
- **Box prompts**: Draw rectangles around target structures

Text-only segmentation is exploratory/assistive, not final-quality.

In [ ]:
# Process first tile (or full image if no tiling needed)
current_tile = tile_paths[0] if tile_paths else INPUT_IMAGE

# Set the image for SAM3
sam3.set_image(str(current_tile))
print(f"Loaded: {current_tile}")

In [ ]:
# Generate masks with text prompt
# This provides initial detection that can be refined interactively
sam3.generate_masks(prompt="building")

In [ ]:
# Save masks as raster
masks_output = OUTPUT_DIR / "masks.tif"
sam3.save_masks(str(masks_output))
print(f"✓ Masks saved: {masks_output}")

In [ ]:
# Visualize results
sam3.show_anns()

In [ ]:
# Show individual masks
sam3.show_masks()

### Interactive Refinement Map

Use the map below to:
1. Enter text prompts (e.g., "building", "house", "garage")
2. Draw bounding boxes around structures
3. Click the **Segment** button to process
4. Refine iteratively until satisfied

In [ ]:
# Launch interactive segmentation map
# - Enter text prompt OR draw bounding box
# - Click "Segment" to process
sam3.show_map(height="700px", min_size=10)

---
## 11. Vectorization & Export

Convert raster masks to vector polygons in Illinois State Plane coordinates.

In [ ]:
def vectorize_and_export(mask_path, output_dir, municipality, min_area_sqft=100, target_crs="EPSG:6455"):
    """
    Convert raster masks to vector polygons with MGP attribute schema.
    
    Parameters:
        mask_path: Path to mask raster
        output_dir: Output directory
        municipality: Municipality name for attributes
        min_area_sqft: Minimum building area filter
        target_crs: Target coordinate system
    
    Returns:
        GeoDataFrame of building footprints
    """
    from rasterio.features import shapes
    from shapely.geometry import shape
    import numpy as np
    
    output_dir = Path(output_dir)
    
    with rasterio.open(mask_path) as src:
        mask = src.read(1)
        transform = src.transform
        src_crs = src.crs
        
        # Extract shapes from mask
        results = (
            {'geometry': shape(geom), 'value': value}
            for geom, value in shapes(mask.astype(np.int32), transform=transform)
            if value > 0  # Exclude background
        )
        
        # Create GeoDataFrame
        geometries = []
        values = []
        for r in results:
            geometries.append(r['geometry'])
            values.append(r['value'])
        
        if not geometries:
            print("⚠ No features extracted from mask")
            return None
        
        gdf = gpd.GeoDataFrame({'mask_id': values}, geometry=geometries, crs=src_crs)
    
    # Reproject to target CRS if needed
    if gdf.crs and gdf.crs.to_epsg() != int(target_crs.split(":")[1]):
        print(f"Reprojecting vectors to {target_crs}...")
        gdf = gdf.to_crs(target_crs)
    
    # Calculate area and filter
    gdf['area_sqft'] = gdf.geometry.area
    initial_count = len(gdf)
    gdf = gdf[gdf['area_sqft'] >= min_area_sqft].copy()
    print(f"Filtered {initial_count - len(gdf)} features below {min_area_sqft} sq ft")
    
    # Add MGP attribute schema
    gdf['building_id'] = range(1, len(gdf) + 1)
    gdf['municipality'] = municipality
    gdf['source'] = 'SAM3_Segmentation'
    gdf['proc_date'] = datetime.now().strftime("%Y-%m-%d")
    gdf['area_sqft'] = gdf['area_sqft'].round(2)
    
    # Reorder columns
    gdf = gdf[['building_id', 'municipality', 'area_sqft', 'source', 'proc_date', 'geometry']]
    
    print(f"\n✓ Extracted {len(gdf)} building footprints")
    print(f"  Total area: {gdf['area_sqft'].sum():,.0f} sq ft")
    print(f"  Mean size: {gdf['area_sqft'].mean():,.0f} sq ft")
    
    return gdf

In [ ]:
# Vectorize masks
buildings_gdf = vectorize_and_export(
    masks_output, 
    OUTPUT_DIR, 
    MUNICIPALITY,
    min_area_sqft=MIN_BUILDING_AREA_SQFT,
    target_crs=TARGET_CRS
)

In [ ]:
# Preview data
if buildings_gdf is not None:
    display(buildings_gdf.head(10))

---
## 12. GIS Export

Export to standard GIS formats for enterprise geodatabase integration.

In [ ]:
def export_to_gis_formats(gdf, output_dir, base_name):
    """
    Export GeoDataFrame to multiple GIS formats.
    
    Exports:
        - GeoPackage (.gpkg) - Recommended for most use cases
        - Shapefile (.shp) - Legacy compatibility
        - GeoJSON (.geojson) - Web/API use
        - File Geodatabase (.gdb) - ArcGIS integration
    """
    output_dir = Path(output_dir)
    
    # GeoPackage (recommended)
    gpkg_path = output_dir / f"{base_name}.gpkg"
    gdf.to_file(gpkg_path, driver="GPKG", layer="building_footprints")
    print(f"✓ GeoPackage: {gpkg_path}")
    
    # Shapefile
    shp_dir = output_dir / "shapefile"
    shp_dir.mkdir(exist_ok=True)
    shp_path = shp_dir / f"{base_name}.shp"
    gdf.to_file(shp_path, driver="ESRI Shapefile")
    print(f"✓ Shapefile: {shp_path}")
    
    # GeoJSON
    geojson_path = output_dir / f"{base_name}.geojson"
    gdf.to_file(geojson_path, driver="GeoJSON")
    print(f"✓ GeoJSON: {geojson_path}")
    
    # File Geodatabase (requires GDAL with FileGDB driver)
    try:
        gdb_path = output_dir / f"{base_name}.gdb"
        gdf.to_file(gdb_path, driver="OpenFileGDB", layer="building_footprints")
        print(f"✓ File Geodatabase: {gdb_path}")
    except Exception as e:
        print(f"⚠ FileGDB export failed (driver may not be installed): {e}")
        print("  Use GeoPackage for ArcGIS Pro import instead")
    
    return {
        'gpkg': gpkg_path,
        'shp': shp_path,
        'geojson': geojson_path
    }

In [ ]:
# Export building footprints
if buildings_gdf is not None and len(buildings_gdf) > 0:
    output_files = export_to_gis_formats(
        buildings_gdf, 
        OUTPUT_DIR, 
        f"{MUNICIPALITY}_buildings_{PROJECT_DATE}"
    )
    
    print(f"\n{'='*60}")
    print("EXPORT COMPLETE")
    print(f"{'='*60}")
    print(f"Municipality: {MUNICIPALITY}")
    print(f"Features: {len(buildings_gdf)}")
    print(f"CRS: {buildings_gdf.crs}")
    print(f"Output: {OUTPUT_DIR}")

---
## 13. Batch Processing (Multiple Tiles)

For large orthophotos, process all tiles and merge results.

In [ ]:
def process_all_tiles(tile_paths, output_dir, municipality, target_crs="EPSG:6455"):
    """
    Process all tiles with SAM3 and merge results.
    
    Note: For production use, interactive refinement should be
    performed on each tile before final export.
    """
    from shapely.ops import unary_union
    
    all_buildings = []
    output_dir = Path(output_dir)
    
    for i, tile_path in enumerate(tile_paths):
        print(f"\nProcessing tile {i+1}/{len(tile_paths)}: {tile_path.name}")
        
        # Set image and generate masks
        sam3.set_image(str(tile_path))
        sam3.generate_masks(prompt="building")
        
        # Save tile masks
        tile_mask = output_dir / "tiles" / f"{tile_path.stem}_mask.tif"
        sam3.save_masks(str(tile_mask))
        
        # Vectorize
        tile_gdf = vectorize_and_export(
            tile_mask, output_dir, municipality,
            min_area_sqft=MIN_BUILDING_AREA_SQFT,
            target_crs=target_crs
        )
        
        if tile_gdf is not None and len(tile_gdf) > 0:
            tile_gdf['source_tile'] = tile_path.name
            all_buildings.append(tile_gdf)
    
    if not all_buildings:
        print("\n⚠ No buildings extracted from any tile")
        return None
    
    # Merge all tiles
    print(f"\nMerging {len(all_buildings)} tile results...")
    merged_gdf = gpd.GeoDataFrame(
        pd.concat(all_buildings, ignore_index=True),
        crs=target_crs
    )
    
    # Reassign building IDs
    merged_gdf['building_id'] = range(1, len(merged_gdf) + 1)
    
    print(f"✓ Merged {len(merged_gdf)} total building footprints")
    return merged_gdf

In [ ]:
# Uncomment to run batch processing on all tiles
# Note: This uses text-only prompts - for production quality,
# use interactive refinement on each tile

# import pandas as pd
# all_buildings_gdf = process_all_tiles(tile_paths, OUTPUT_DIR, MUNICIPALITY, TARGET_CRS)
# if all_buildings_gdf is not None:
#     export_to_gis_formats(all_buildings_gdf, OUTPUT_DIR, f"{MUNICIPALITY}_all_buildings_{PROJECT_DATE}")

---
## 14. Quality Assurance Checklist

Before delivering to client:

- [ ] **CRS Verification**: Confirm EPSG:6455 (Illinois State Plane East ftUS)
- [ ] **Geometry Validation**: No self-intersections, slivers, or multipart where inappropriate
- [ ] **Attribute Completeness**: All required fields populated
- [ ] **Visual QC**: Overlay on source imagery in ArcGIS Pro/QGIS
- [ ] **Edge Matching**: Check tile boundaries for duplicate/split buildings
- [ ] **Area Validation**: Spot-check building areas against known dimensions
- [ ] **File Integrity**: Verify all export formats open correctly

In [ ]:
def qa_check(gdf):
    """Run basic QA checks on building footprints."""
    print("QA CHECKS")
    print("=" * 40)
    
    # CRS check
    print(f"CRS: {gdf.crs}")
    if gdf.crs.to_epsg() == 6455:
        print("  ✓ Correct CRS (EPSG:6455)")
    else:
        print("  ⚠ CRS mismatch - should be EPSG:6455")
    
    # Geometry validity
    invalid = ~gdf.is_valid
    if invalid.sum() == 0:
        print(f"  ✓ All {len(gdf)} geometries valid")
    else:
        print(f"  ⚠ {invalid.sum()} invalid geometries found")
    
    # Attribute completeness
    null_counts = gdf.isnull().sum()
    if null_counts.sum() == 0:
        print("  ✓ No null values in attributes")
    else:
        print(f"  ⚠ Null values found: {null_counts[null_counts > 0].to_dict()}")
    
    # Area statistics
    print(f"\nArea Statistics (sq ft):")
    print(f"  Min: {gdf['area_sqft'].min():,.0f}")
    print(f"  Max: {gdf['area_sqft'].max():,.0f}")
    print(f"  Mean: {gdf['area_sqft'].mean():,.0f}")
    print(f"  Median: {gdf['area_sqft'].median():,.0f}")

if buildings_gdf is not None:
    qa_check(buildings_gdf)

---
## Notes

### Common Issues

**Small buildings missed**: Reduce tile size to 1024×1024 px

**Over-segmentation**: Use interactive box prompts instead of text

**Merge artifacts at tile edges**: Use overlapping tiles and dissolve duplicates

**Memory errors**: Process on GPU with at least 8GB VRAM, or reduce tile size

### Resources

- [segment-geospatial Documentation](https://samgeo.gishub.org/)
- [SAM3 on Hugging Face](https://huggingface.co/facebook/sam3)
- [Cook County GIS](https://hub-cookcountyil.opendata.arcgis.com/)
- [EPSG:6455 Reference](https://epsg.io/6455)

---

*MGP Building Segmentation Pipeline v1.0*  
*Municipal GIS Partners • Spark Initiative*